<span style="font-family:Times New Roman; font-size:2.0em;">**Supplement 04**</span>

One of the places that computers shine most is in automating repetitive tasks. When faced with calculations over large sets, we frequently handle them using loops. To truly appreciate the benefit of a loop, you have to think big - if you need to do one or ten iterations to get an answer, then it is probably more efficient to solve your problem by hand, using a calculator, or using a spreadsheet. But when you have to perform an operation hundreds or thousands of times, then you'll want to use a programming language, and you will probably consider looping those calculations to automate iterations so that you don't have to write them out line by line.

We will use two types of loops: **for loops** and **while loops**. At a high level, they are similar in that they will repeat an operation in a cyclic fashion until they meet some criteria that we have specified, at which point, they terminate and exit, returning any outputs that we generated over the coures of the loop.

The fundamental difference between the two types of loops is this: **For loops** will cyclicly perform an operation on every element in a pre-defined set. **While loops** will cyclically perform an operation until threshold criteria are met.

As usual, the best way to get a feel for this is with some examples.  

**An important note before we start:**  for loops and while loops are really just general tools that you can use in solving problems and processing data. Both are almost infinitely flexible in how they are used. It is important to remember this in going through the next examples. These will illustrate some examples of using loops, but they are intended to get you familiar with their architechure, syntax, logic, and execution. Do not get stuck in thinking "This is always how I should use a for loop" or "I can only use a while loop in this situation." You really can abstract both types of loops to solve many different problems, it only requires that you think about how to adapt the basic architecture of the loop to solve the problem at hand.

<span style="font-family:Times New Roman; font-size:1.5em;">**For Loops**</span>

Let's start with a basic for loop.  Again, the key thing about a for loop is that it will perform the specified operation for the number of times that we indicate. For example, let's say I have a 1D Array that contains evenly spaced integers between 2 and 20.

    A = [2,4,6,8,10,12,14,16,18,20]

Let's just write a for loop to print out a statement that indicates the current value of the array element. To do that, I will use a classic, Matlab-style approach. I'll initiate a for loop by defining the number of times I want it to iterate.  Here, I know I want to go from the first iteration to the total number of elements that are in A.  Then, for each iteration, I will print the current value of the index, i, and the number in A, A(i).

    using Printf             #Need to add the formatted printing module
    for number in A
        @printf("The current number is %d\n", number)
    end

This loop will just print the "current" value of A as the loop iterates through each element in A. You must end a for loop with an **end** keyword in Julia

**Note:** *@printf* is a julia macro that enables formatted printing.  I find it has good compatibility with Matlab syntax.  See link below 

https://www.mathworks.com/help/simulink/slref/fprintf.html

There are probably an infinite number of ways to construct and use a for loop. In very general terms, the thing that I do most commonly with for loops is to use the iterable as an index to reference elements in data sets for operations in a for loop.

If I wanted to do some operation on all of the elements in A, it is easy to execute in a for loop.

    A = [1,2,3,4,5,6,7,8,9,10,11]
    for i = 1:length(A)
        B = A[i]*7
        @printf("The current value of the index is %d, and the current number in A multiplied by 7 is %d\n", i, B)
    end

As another example, imagine we have a set of 8 measurements that we made experimentally, we'll call them **ex**.  Just to ground this, let's say these are 8 mass transfer coefficients that we measured at 8 different liquid velocities. They have units of cm/sec.

    ex = [10.75, 12.52, 17.90, 18.4, 22.36, 35.10, 56.2, 74.95] 
    
As we often do in engineering, we are developing a model that is supposed to predict the value of each of those mass transfer coefficients at a given fluid velocity. Without getting into the specifics of that model, let's just say that our model predicts that those 8 mass transfer coefficients at those same 8 fluid velocities have the following values, which we store in **mod**.

    mod = [9.4, 13.62, 15.89, 19.23, 21.75, 36.24, 59.46, 73.13]
    
To figure out how well our model agrees with our measurements, we generally calculate the residual square error, i.e., the difference between the model's prediction and the measurement's value (squared). I see that I have two lists, both of which are 8 elements long. I want to subtract the values in these lists element by element, and I want to square that result.  The conventional way to do this in most languages is with a for loop:

    SE = zeros(length(ex),) #create an empty array to store residual values
    for k = 1:length(ex)
        SE[k] = (ex[k] - mod[k])^2
    end
    SE

This for loop simply calculates and stores the difference between model and experiment (squared) for each condition tested, but this general concept is easily abstracted and applied to much more complex problems.



If you are solving a least squares problem, the next thing you'd probably do is calculate the sum of square error (SSE).  I could do that by taking advantage of the sum() function in Julia, which sums the elements in a specified array.

    SSE = sum(SE)

It is worth pointing out that Julia has the ability to perform element-wise operations on matrices and vectors, which can very often replace a for loop with much more concise syntax.  For example, to calculate a vector of residuals, we could simply write:

    residual = ex - mod #Subtracting two (8,) arrays element-by-element is no problem.

We can square each element in residual using an element-wise operator in front of the ^:

    SE = residual.^2
    
And we can sum the result:

    SSE = sum(SE)
    
Or in a single line:

    SSE = sum((ex - mod).^2)
    
You get the same result since this gives an element-by-element operation. Generally, I would use this approach since it is much neater and more efficient, but sometimes, you will need to write a for loop to perform an iterative operation, and these simple examples are a good way to learn how to use them.

Circling back to that comment I made about loops really shining when you need to repeat an operation over a large set. Here's an example that is relevant to chemistry and engineering. This example requires a bit of background in thermodynamics, but it should be at least somewhat familiar if you've taken general chemistry, finished material and energy balances, or heard of "Hess's Law". 

**Context**

Let's say I am in the proces engineer in charge of a Fluidized Catalytic Cracker (FCC). An FCC unit is something everyone should at least know about - it is type of reactor used extensively in an oil refinery. Crude oil contains a lot of heavy, high molecular weight species that are not much use as fuels or chemical feedstocks. An FCC unit "cracks" these high molecular weight hydrocarbons to make gasoline and light alkenes (ethylene, propylene, butene, butadiene), which are far more valuable. There are hundreds, if not thousands of chemical reactions happening inside of a catalytic cracker. For the purpose of this example, let's say there are...100 different reactions.

Each of these 100 reactions has a unique enthalpy change (DH), a unique entropy change (DS), and a unique Gibbs free energy change (DG) associated with it. The enthalpy change, entropy change, and temperature determine the Gibbs free energy change according to the very simple equation below:

$$\Delta G = \Delta H - T\Delta S$$

We then use the Gibbs free energy change to calculate the equilibrium constant (K) for each reaction using another very simple equation. 

$$K = e^\left(\frac{-\Delta G}{RT}\right)$$

Where **e** is the standard exponential function and R is the gas constant in appropriate units. The equilibrium constant is important because it allows us to determine the composition of our reacting mixture at chemical equilibrium. That seems like something that would be really important for any chemical process, so why not catalytic cracking? To do that for an FCC unit, we would need to calculate all 100 equilibrium constants - there is one unique K for each reaction. 

That sounds like a ton of tedious work to me. But it turns out that we can easily do this with for loops (or element-wise operations). The worst part, by far, is finding the thermodynamic data for these reactions. We are going to take the lazy approach for finding thermodynamic data, which is to make it up completely for the purpose of solving this example. To this end, I will use a random integer generator to create a list of 100 *reasonable* enthalpy changes of reaction and 100 *reasonable* entropy changes of reaction. **Professor's note: This will never fly in your thermo or mass and energy courses, please understand that I am just doing this to demonstrate how to use a for loop to process a large data set.**

    DS = rand(-75:0.1:75, (100,))         #units all in J/mol/K
    DH = rand(-100000:0.1:100000, (100,)) #units all in J/mol

Now that the very difficult work of finding enthalpies and entropies of reaction is done, we can automate application of those two simple formulas above to calculate the equilibrium constant for each reaction:

$$\Delta G = \Delta H - T\Delta S$$
$$K = e^\left(\frac{-\Delta G}{RT}\right)$$

For now, we'll do this at a temperature of 500K. Working in units of Joules, moles, and K, the gas constant has a value of 8.314 J/mol/K. In engineering, you'll never regret keeping careful track of units!

I have 100 elements in $\Delta S$ and $\Delta H$, and I want to perform the calculations for each of those elements to calculate 100 different equilibrium constants, so I want my index to go from from 1 to 100 (the length of DH). Then I can use those numbers as indices to perform operations on each element.
    
    T = 500                 #K
    R = 8.314               #J/mol/K
    K = zeros(length(DH),)  #create empty space to store K's
    for i = 1:length(DH)
        DG   = DH[i] - T*DS[i]
        K[i] = exp(-DG/R/T)
    end
    K

That's pretty sweet! A few lines of code, it runs in less than a second and we're all done! By far, the most difficult part of this in the real world would be the part that we skipped -- finding accurate thermodynamic data (DH and DS) for 100 different reactions!

It is again worth pointing out that, in this particular example, a for loop is not necessary since you can take advantage of element-wise operations in Julia:
    
    DG = DH - T*DS
    K  = exp.(-DG/R/T)   #Note, I'm broadcasting with exp(), otherwise it will fail because it isn't natively vectorized.

But maybe that doesn't quite impress you. I get it. You're a die hard Excel fan and you know that, realistically, if you just made up 100 enthalpies and entropies of reaction too, it would be easy enough to calculate 100 equilibrium constants at 500K by autofilling the cells. Fair point. *For a single temperature.*  But what if you wanted to know how each of those equilibrium constants changed as smooth functions of temperature between, say, 300K and 900K? That's a pretty big span...Let's calculate those equilibrium constants at 300K, 302K, 304K, etc. all the way up to 900K to generate sets of equilibrium constants as a function of Temperature for each of our 100 reactions.

Although it isn't impossible, it gets unwieldy in a spreadsheet, but as we see below, I only have to modify my script slightly to do this in Octave.

First, I will create a set of the temperatures I'm interested in. I want to take steps of 2K and go from 300 to 900K. I'll just use a range constructor to create a column of temperatures. 

    T = 300:2:900

Now, we have to do a bit of abstraction and see if we can generalize our last for loop to run not just on one temperature, but on all 301 temperatures. I will do this with pair of for loops, one nested inside of the other. Notice that when I create nested for loops, each **for** keyword must be closed with a matching **end** keyword.

The logic:  I want to run my original for loop for every Temperature in the list of temperatures we just created. So, I will create two for loops.  The first one will specify the current temperature, and the second will specify the current reaction. Ultimately I will store all of my equilibrium constants in a 2D matrix, so I pre-allocate something of the correct size before starting the loop:

    K = zeros(length(T), length(DH)) 
    for i = 1:length(T)    
        for j = 1:length(DH)                
            DG = DH[j] - T[i]*DS[j]   
            K[i,j] = exp.(-DG/R/T[i])   
        end
    end
    size(K)
                                  
                                  
The end result here is going to be 100 different equilibrium constants calculated at 301 different temperatures, i.e., a 301x100 set of equilibrium constants. It is a matrix of equilibrium constants. Each column (j) in the matrix is the complete set of equilibrium constants for reaction (j) at all temperatures. Each row (i) in the matrix represents the complete set of equilibrum constants for all reactions at each temperature (i). For example, $K[:,1]$ is the complete set of equilibrium constansts for the first reaction at all 301 temperatures, $K[:,2]$ is the analogous set for reaction 2, etc., whereas $K(1,:)$ is the complete set of equilibrium constants for all reactions at 300K, $K(end,:)$ is the complete set of equilibrium constants for all reactions at 900K, etc. 

I do not suggest printing K.  At this point, the best way to visualize the data is graphically; we will introduce this briefly below.

In Julia, we have to first add a plotting package and then declare that we are using Plots, but then, we can create a basic plot using a similar syntax to Matlab.  First, the installation.  I recommend doing this from Julia in terminal, not in a Notebook, but it should work either way.

    using Pkg
    Pkg.add("Plots")

Those two commands will install the plots package.

Now that we have Plots installed, it's pretty straightforward to create a basic line or scatterplot:

    using Plots
    plot(independent_variable,dependent_variable, *options*)
    scatter(independent_variable, dependent_variable, *options*)
    
There is a pretty comprehensive plotting tutorial, including formatting options here:

https://docs.juliaplots.org/latest/tutorial/

Let's say we want to plot the equilibrium constant for the first reaction as a function of temperature.  Our equilibrium constants are stored in a matrix.  The matrix has 301 rows, and each row has 100 columns list that contains the equilibrium constants we calculated at each temperature.  So:

    K[:,1] 
    
Will extract the equilibrium constant for the first reaction at all 301 temperatures;

    K[:,10]
    
Will extract the equilibrium constant for the 10th reaction at all 301 temperatures, etc.

So, if I wanted to create an inline plot the equilibrium constant for the first reaction as a function of temperature using, I could write:
    
    plot(T,K[:,1])

For the record, this is not the best way to plot K = f(T).  The thing about equilibrium constants is that they are such a strong function of temperature that they change over orders of magnitude.  This sounds like a job for a semilog plot.  I'm also going to start adding formatting options so that you can see how this is done.  Note that in these options series "Label" refers to what you call each data set and will appear in a legend.  I'm turning it off since I don't want a legend here.

    plot(T, K[:,1], yaxis = :log10, xlabel = "T(K)", ylabel = "K", legend = nothing)
    

Much better. 

Julia will generally re-plot everytime you invoke a new plot command. If I want to plot multiple equilibrium constants, I can do so in a few ways:

I can use a bang (!) to tell it to update the plot we've already made (similar to append!() and sort!():


    plot(T, K[1,:], yaxis = :log10, xlabel = "T(K)", ylabel = "K", legend = nothing)
    plot!(T, K[2,:])

Well, of course now that we went and generated a 301 x 100 data set, I would really like to plot **all** 100 equilibrium constants as a function of temperature, but I really do not want to write a separate plot! command for every element in that matrix. Julia is nice in that it allows you to plot many different dependent variables (K) for a single set of independent variables (T) in a single line; you just have to make sure that the dimensions are consistent.  Here, we are plotting a 301 element temperature vector against each row in K, which has 301 columns:

    plot(T, K, yaxis = :log10, xlabel = "T(K)", ylabel = "K", legend = nothing)
    println(maximum(K))
    println(minimum(K))
    
I added the last two lines just to get a readout of the smallest and largest equilibrium constants in this set.  We'll compare with an alternate method below.

I'll point out one more thing that I like about Julia.  Again, although I can use a for loop to do what I did here (as above), Julia has really robust element-wise operations, and I can just as easily calculate this 301x100 matrix of equilibirum constants using them:

    DG = transpose(DH - T'.*DS);
    K  = exp.(-DG/R./T);
    plot(T, K, yaxis = :log10, xlabel = "T(K)", ylabel = "K")
    println(maximum(K))
    println(minimum(K))

This absolutely the same result that you get with the two nested for loops above and is, in my opinion, the better way to handle this problem in Octave. 

You should get used to working with both loops and vectorized, element-by-element operations.  Loops are often easier to visualize logically since element-wise operations require you to pay very close attention to dimensions of your arrays, but element-wise operations are more efficient and easier to read.  Both have their place, so familiarize yourself with them

<sup>3</sup> Anastasio, E.J. and Marshall, T.